Chat GPT's code for input->circuit

In [1]:
class Gate:
    def __init__(self, gate_type, inputs, output):
        self.gate_type = gate_type
        self.inputs = inputs
        self.output = output


def parse_input(input_str):
    lines = input_str.strip().split('\n')
    
    primary_inputs = lines[0].split()[1:]
    primary_outputs = lines[1].split()[1:]
    internal_signals = lines[2].split()[1:]
    
    gates = []
    for line in lines[3:]:
        tokens = line.split()
        gate_type = tokens[0]
        inputs = tokens[1:-1]
        output = tokens[-1]
        gates.append(Gate(gate_type, inputs, output))
    
    return primary_inputs, primary_outputs, internal_signals, gates


def generate_circuit(primary_inputs, primary_outputs, internal_signals, gates):
    circuit = ""
    
    circuit += "Inputs: {}\n".format(", ".join(primary_inputs))
    circuit += "Outputs: {}\n".format(", ".join(primary_outputs))
    
    for gate in gates:
        inputs = ", ".join(gate.inputs)
        circuit += "{}: {} -> {}\n".format(gate.gate_type, inputs, gate.output)
    
    return circuit


input_spice = """
PRIMARY_INPUTS A B
PRIMARY_OUTPUTS E F
INTERNAL_SIGNALS C D
INV A C
AND2 C B D
NAND2 C D E
INV D F
"""

primary_inputs, primary_outputs, internal_signals, gates = parse_input(input_spice)
circuit_representation = generate_circuit(primary_inputs, primary_outputs, internal_signals, gates)

print(circuit_representation)


Inputs: A, B
Outputs: E, F
INV: A -> C
AND2: C, B -> D
NAND2: C, D -> E
INV: D -> F



Chat GPT-> for time delay

In [44]:
class Gate: # struct gate
    def __init__(self, gate_type, inputs, output):
        self.gate_type = gate_type
        self.inputs = inputs
        self.output = output

    def print(self):
        print(f"This is a {self.gate_type}\nThe inputs to this gate are {self.inputs}\nThe output of this gate is {self.output}")

def parse_input(input_str):
    lines = input_str.strip().split('\n')
    
    primary_inputs = lines[0].split()[1:]
    primary_outputs = lines[1].split()[1:]
    internal_signals = lines[2].split()[1:]
    
    gates = []
    for line in lines[3:]:
        tokens = line.split()
        gate_type = tokens[0]
        inputs = tokens[1:-1]
        output = tokens[-1]
        gates.append(Gate(gate_type, inputs, output))
    
    return primary_inputs, primary_outputs, internal_signals, gates


def parse_gate_delays(gate_delay_str):
    gate_delays = {}
    lines = gate_delay_str.strip().split('\n')
    for line in lines:
        tokens = line.split()
        gate_type = tokens[0]
        delay = float(tokens[1])
        gate_delays[gate_type] = delay
    return gate_delays

def compute_output_delay(gate, gate_map, gate_delays, delay_cache):
    if gate.output in delay_cache:
        return delay_cache[gate.output]

    max_input_delay = 0
    for input in gate.inputs:
        input_gate = input.gate_type
        input_delay = compute_output_delay(input_gate, gate_map, gate_delays, delay_cache)
        max_input_delay = max(max_input_delay, input_delay)
    
    gate_delay = gate_delays[gate.gate_type]
    output_delay = max_input_delay + gate_delay
    delay_cache[gate.output] = output_delay
    return output_delay

def compute_output_delays(primary_outputs, gates, gate_map, gate_delays):
    output_delays = {}
    delay_cache = {}

    for output_signal in primary_outputs:
        output_gate = gate_map[output_signal]
        output_delay = compute_output_delay(output_gate, gate_map, gate_delays, delay_cache)
        output_delays[output_signal] = output_delay
    
    return output_delays

input_spice = """
PRIMARY_INPUTS A B
PRIMARY_OUTPUTS E F
INTERNAL_SIGNALS C D
INV A C
AND2 C B D
NAND2 C D E
INV D F
"""

gate_delay_str = """
NAND2 3
AND2 4
NOR2 3.5
OR2 4.5
INV 2
"""

primary_inputs, primary_outputs, internal_signals, gates = parse_input(input_spice) # gates => struct of each gate
gate_delays = parse_gate_delays(gate_delay_str) # dict of gates and time delays
gate_map = {gate.output: gate for gate in gates}

output_delays = compute_output_delays(primary_outputs, gates, gate_map, gate_delays)

# Print or write the output delays to a file
for output_signal, delay in output_delays.items():
    print("Primary Output:", output_signal)
    print("Delay:", delay)


TypeError: 'Gate' object is not iterable

My Implementation

In [48]:
class Gate:
    def __init__(self, gate_type, delay, inputs, output):
        self.gate_type = gate_type
        self.delay = delay
        self.inputs = inputs
        self.output = output
    
    def print_attr(self):
        print(f"This is a {self.gate_type}\nThe inputs to this gate are {self.inputs}\nThe output of this gate is {self.output}\nThe Delay for the gate is {self.delay}\n")

    def print(self):
        print(f"{self.inputs} --> {self.gate_type} in {self.delay}sec --> {self.output} ")
        
def find_prev(gates, input_signal, primary_inputs) -> Gate:
    # input_signal is a single string that consists of one of the input signals
    # gates is the list of Gate(s), gate.output is a single string that gives the output of the gate
    for gate in gates:
        if check_primary_input(primary_inputs, input_signal) and (input_signal == gate.output):
            return gate
    
def check_primary_input(primary_inputs, input_signal) -> bool:
    if (input_signal in primary_inputs):
        return True
    return False

def connect_two_nodes(gate, gates, primary_inputs):
    for i, input in enumerate(gate.inputs):
        if (not check_primary_input(primary_inputs, input)):
            prev_gate = find_prev(gates, input, primary_inputs)
            new_input = gate.inputs[:i] + gate.inputs[i+1:] + prev_gate.inputs # new inputs list

def parse_input(input_spice, gate_delay_str):
    lines = input_spice.strip().split('\n') # list that breaks the input into elements 
    delays_list = gate_delay_str.strip().split('\n') # list that breaks the delays into elements
    delays_dict = {} # dict for later use {gate_type: delay}
    for gate_delays in delays_list:
        temp = gate_delays.split(" ")
        gate = temp[0]
        delay = temp[1]
        delays_dict[gate] = float(delay)

    primary_inputs = lines[0].split()[1:]
    primary_outputs = lines[1].split()[1:]
    internal_signals = lines[2].split()[1:]
    
    gates = []  # array of Gate(s)
    for line in lines[3:]:
        tokens = line.split()
        gate_type = tokens[0]
        inputs = tokens[1:-1]
        output = tokens[-1]
        gates.append(Gate(gate_type, delays_dict[gate_type], inputs, output))
    
    # primary_inputs is an array of primary inputs
    # primary_outputs is an array of primary outputs
    # internal_signals is an array of internal signals
    # the array of gates
    return primary_inputs, primary_outputs, internal_signals, gates


input_spice = """
PRIMARY_INPUTS A B
PRIMARY_OUTPUTS E F
INTERNAL_SIGNALS C D
INV A C
AND2 C B D
NAND2 C D E
INV D F
"""

gate_delay_str = """
NAND2 3
AND2 4
NOR2 3.5
OR2 4.5
INV 2
"""

primary_inputs, primary_outputs, internal_signals, gates = parse_input(input_spice, gate_delay_str)

for gate in gates:
    # gate.print_attr()
    gate.print()

['E', 'F']